In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import utils.dataloaders as dl

import utils.single_maha as maha

import model_params as params
import model_paths

from importlib import reload

In [4]:
reload(maha)
reload(params)
reload(model_paths)
device = torch.device('cuda:0')

datasets = ['MNIST', 'FMNIST', 'SVHN', 'CIFAR10']

#datasets = ['CIFAR10']

for dataset in datasets:
    print(dataset)
    model_params = params.params_dict[dataset](augm_flag=True)
    path = model_paths.model_dict[dataset]().file_dict['Base']

    pretrained = torch.load(path).state_dict()

    if dataset=='MNIST':
        model = maha.LeNetMadry()
    elif dataset=='FMNIST':
        model = maha.ResNet18(1, 10)
    elif dataset in ['SVHN', 'CIFAR10']:
        model = maha.ResNet18(3, 10)
    elif dataset=='CIFAR100':
        model = maha.ResNet18(3, 100)

    model.load_state_dict(pretrained)

    maha_model = maha.Mahalanobis(model.to(device), model_params, device)
    final_model = maha.ModelODIN(maha_model, model_params, device)

    torch.save(final_model.cpu(), 'SavedModels/other/single_mahalanobis/' + dataset + '.pth')

MNIST
FMNIST
SVHN
CIFAR10


In [3]:
data, label = next(iter(dl.MNIST(train=False)))


In [13]:
import utils.eval as ev

dataset = 'CIFAR10'

model_params = params.params_dict[dataset](augm_flag=True)
device = torch.device('cuda:0')

model = torch.load('SavedModels/other/single_mahalanobis/' + dataset + '.pth').to(device)

In [17]:
out = ev.evaluate(model, device, dataset, model_params.loaders, load_adversaries=False, 
                  writer=None, epoch=0, drop_mmc=True)

In [18]:
out

,AUC
DataSet,
SVHN,94.834108
CIFAR100,68.692244
LSUN_CR,68.389567
Imagenet-,65.379701
Noise,81.338357
UniformNoise,56.760514


In [24]:
out.show()

AttributeError: 'DataFrame' object has no attribute 'show'

In [21]:
model.epsilon = 0.005
ev.evaluate(model, device, dataset, model_params.loaders, load_adversaries=False, 
                  writer=None, epoch=0, drop_mmc=True)

,AUC
DataSet,
SVHN,65.959964
CIFAR100,66.836739
LSUN_CR,74.772767
Imagenet-,74.329027
Noise,59.163790
UniformNoise,13.481253


In [25]:
epsilons = [0.0, 0.01, 0.005, 0.002, 0.0014, 0.001, 0.0005]

for eps in epsilons:
    model.epsilon = eps
    df = ev.evaluate(model, device, dataset, model_params.loaders, load_adversaries=False, 
                  writer=None, epoch=0, drop_mmc=True)
    print(df)
    print('')

                    AUC
DataSet                
SVHN          83.914293
CIFAR100      83.246608
LSUN_CR       88.965033
Imagenet-     85.759728
Noise         93.687199
UniformNoise  98.306508

                    AUC
DataSet                
SVHN          55.869605
CIFAR100      65.565038
LSUN_CR       76.273233
Imagenet-     76.221443
Noise         49.215109
UniformNoise  24.182202

                    AUC
DataSet                
SVHN          65.959979
CIFAR100      66.836745
LSUN_CR       74.772567
Imagenet-     74.329019
Noise         59.163777
UniformNoise  13.481271

                    AUC
DataSet                
SVHN          79.926930
CIFAR100      61.086927
LSUN_CR       63.244833
Imagenet-     63.868304
Noise         72.635824
UniformNoise  14.156658

                    AUC
DataSet                
SVHN          85.167535
CIFAR100      60.377637
LSUN_CR       60.628800
Imagenet-     61.100651
Noise         77.364653
UniformNoise  19.389752

                    AUC
DataSet    